In [1]:
import ijson
import json
import csv
import decimal
import string

def parse_tweets(infile, hdict, eco):
    ht_dict = hdict["hashtags"]
    ht_id_counter = len(ht_dict) + 1
    
    url_dict = hdict["urls"]
    url_id_counter = len(url_dict) + 1
    
    user_dict = hdict["users"]
    tweet_dict = hdict["tweets"]
    edges_dict = hdict["edges"]

    f = open(infile)
    # parse the tweet
    tweets = ijson.items(f, "item")  
    hasTweets = True
    while(hasTweets):
        try:
            tweet = next(tweets)
            created_at = tweet["created_at"]  
            try:
                if tweet["truncated"] == True and 'extended_tweet' in tweet and 'full_text' in tweet['extended_tweet']:
                            text = tweet["extended_tweet"]['full_text'].replace("\\", "").replace("\n", " ").replace("\t", " ")
                else:
                    text = tweet["text"].replace("\\", "").replace("\n", " ").replace("\t", " ").replace("\t", " ")
            except KeyError:
                continue                    
            tid = str(tweet["id"])

            user_id = str(tweet["user"]["id"])
            screen_name = tweet["user"]["screen_name"]
            user_name = tweet["user"]["name"]
            user_dict[user_id] = [screen_name, user_name, eco]                  

            lang = tweet["lang"]
            likes = tweet["favorite_count"]

            place = tweet["place"]        
            country_code = place["country_code"] if place and "country_code" in place else ""
            country = place["country"] if place and "country" in place else ""
            full_name = place["full_name"] if place and "full_name" in place else ""
            name = place["name"] if place and "name" in place else ""
            coordinates = place["bounding_box"]["coordinates"] if place else "" 

            hashtags = tweet["entities"]["hashtags"]
            user_mentions = tweet["entities"]["user_mentions"]
            urls = tweet["entities"]["urls"]
            media = tweet["entities"]["media"] if "media" in tweet["entities"] else []

            #reply tweets
            tid_replies_to = tweet["in_reply_to_status_id"]

            #retweetes 
            rt_id = ""
            rt_status = tweet.get("retweeted_status", "")
            if rt_status:
                rt_created_at = rt_status["created_at"] 
                rt_id = str(rt_status["id"])            
                rt_user_id = str(rt_status["user"]["id"])
                rt_text = rt_status["text"].replace("\\", "").replace("\n", " ").replace("\t", " ")

                if rt_status["truncated"] == True:
                    if 'extended_tweet' in rt_status and 'full_text' in rt_status['extended_tweet']:
                        rt_text = rt_status["extended_tweet"]['full_text'].replace("\\", "").replace("\n", " ").replace("\t", " ")

                rt_lang = rt_status["lang"]
                rt_likes = rt_status["favorite_count"]
                rt_place = tweet["place"]        
                rt_country_code = rt_place["country_code"] if rt_place and "country_code" in rt_place else ""
                rt_country = rt_place["country"] if rt_place and "country" in rt_place else ""
                rt_full_name = rt_place["full_name"] if rt_place and "full_name" in rt_place else ""
                rt_name = rt_place["name"] if rt_place and "name" in rt_place else ""
                rt_coordinates = rt_place["bounding_box"]["coordinates"] if rt_place else ""      

                rt_row = [rt_id, rt_lang, rt_name, rt_text, rt_created_at, rt_full_name, rt_country, 
                          rt_country_code,rt_coordinates, rt_likes, "tweet", eco]

                if  rt_id not in tweet_dict:
                    tweet_dict[rt_id] = rt_row
                    edges = [rt_user_id, rt_id, "TWEETS", eco]
                    edges_dict['_'.join(edges)] = edges                       
                elif tweet_dict[rt_id][3] == "":
                    tweet_dict[rt_id] = rt_row

                rt_user_screen_name = rt_status["user"]["screen_name"]
                rt_user_name = rt_status["user"]["name"]
                user_dict[rt_user_id] = rt_user_screen_name, rt_user_name, eco

            #quoted tweets    
            q_id = ""
            q_status = tweet.get("quoted_status", "")
            if q_status:
                q_created_at = q_status["created_at"] 
            
                q_id = str(q_status["id"])
                q_user_id = str(q_status["user"]["id"])
                q_text = q_status["text"].replace("\\", "").replace("\n", " ").replace("\t", " ")
                if q_status["truncated"] == True:
                    if 'extended_tweet' in q_status and 'full_text' in q_status['extended_tweet']:
                        q_text = q_status["extended_tweet"]['full_text'].replace("\\", "").replace("\n", " ").replace("\t", " ")


                q_lang = q_status["lang"]
                q_likes = q_status["favorite_count"]
                q_place = q_status["place"]               
                q_country_code = q_place["country_code"] if q_place and "country_code" in q_place else ""
                q_country = q_place["country"] if q_place and "country" in q_place else ""
                q_full_name = q_place["full_name"] if q_place and "full_name" in q_place else ""
                q_name = q_place["name"] if q_place and "name" in q_place else ""
                q_coordinates = q_place["bounding_box"]["coordinates"] if q_place else ""    

                q_row = [q_id, q_lang, q_name, q_text, q_created_at, q_full_name, q_country, 
                          q_country_code, q_coordinates, q_likes, "tweet", eco]

                if  q_id not in tweet_dict:
                    tweet_dict[q_id] = q_row
                    edges = [q_user_id, q_id, "TWEETS", eco]
                    edges_dict['_'.join(edges)] = edges                     
                elif tweet_dict[q_id][3] == "":
                    tweet_dict[q_id] = q_row

                q_user_screen_name = q_status["user"]["screen_name"]
                q_user_name = q_status["user"]["name"]
                user_dict[q_user_id] = q_user_screen_name, q_user_name, eco

            row = [tid, lang, name, text, created_at, full_name, country, country_code, coordinates, likes, "tweet", eco]

            if tid not in tweet_dict:
                tweet_dict[tid] = row
                edges = [user_id, tid, "TWEETS", eco]
                edges_dict['_'.join(edges)] = edges
            elif tweet_dict[tid][3] == "":
                tweet_dict[tid] = row

            # write out edges
            if rt_id:
                edges = [tid, rt_id, "RETWEETS", eco]
                edges_dict['_'.join(edges)] = edges

            if tid_replies_to:
                tid_replies_to = str(tid_replies_to)

                if tid_replies_to not in tweet_dict:
                    tweet_dict[tid_replies_to] = [tid_replies_to, "", "", "", "", "","","","","", "tweet", eco]
                edges = [tid, tid_replies_to, "REPLIES_TO", eco]
                edges_dict['_'.join(edges)] = edges
                id_user_replies  = tweet["in_reply_to_user_id_str"]

                if id_user_replies not in user_dict:
                    screen_name_user_replies = tweet["in_reply_to_screen_name"]
                    user_dict[id_user_replies] = screen_name_user_replies, "", eco
                edges = [id_user_replies, tid_replies_to, "TWEETS", eco]
                edges_dict['_'.join(edges)] = edges

            if q_id:
                edges = [tid, q_id, "QUOTED_TO", eco]
                edges_dict['_'.join(edges)] = edges


            for hashtag in hashtags:
                hashtag = hashtag["text"].lower()
                if hashtag not in ht_dict:
                    ht_id = "h{}".format(ht_id_counter)
                    ht_dict[hashtag] = [ht_id, eco]
                    ht_id_counter += 1
                else:
                    ht_id = ht_dict[hashtag][0]
                edges = [tid, ht_id, "USES", eco]
                edges_dict['_'.join(edges)] = edges    

            for url in urls:
                url = url["expanded_url"]
                if url not in url_dict:
                    url_id = "u{}".format(url_id_counter)
                    url_dict[url] = [url_id, eco]
                    url_id_counter += 1
                else:
                    url_id = url_dict[url][0] 
                edges = [tid, url_id, "LINKS", eco]
                edges_dict['_'.join(edges)] = edges      

            for m in media:
                m = m["expanded_url"]
                if m not in url_dict:
                    url_id = "m{}".format(url_id_counter)
                    url_dict[m] = [url_id, eco]
                    url_id_counter += 1
                else:
                    url_id = url_dict[m][0] 
                edges = [tid, url_id, "LINKS", eco]
                edges_dict['_'.join(edges)] = edges     

            for user_mention in user_mentions:
                uid = str(user_mention["id"])
                mention_screen_name = user_mention["screen_name"]
                mention_user_name = user_mention["name"]            
                user_dict[uid] = mention_screen_name, mention_user_name, eco
                edges = [tid, uid, "MENTIONS", eco]
                edges_dict['_'.join(edges)] = edges  
       
        except StopIteration:
            hasTweets = False
        except: 
            continue
    
    hdict = {}
    hdict['hashtags'] = ht_dict
    hdict["users"] = user_dict
    hdict["urls"] = url_dict
    hdict["tweets"] = tweet_dict
    hdict["edges"] = edges_dict
    return hdict

In [25]:
def create_csvs(file, hdict):

    tweetfile = open(file + "tweets.csv", "w")
    tweet_writer = csv.writer(tweetfile, delimiter='\t')
    
    edgefile = open(file + "edges.csv", "w")
    edge_writer = csv.writer(edgefile, delimiter='\t')

    tagfile = open(file + "tags.csv", "w")
    tag_writer = csv.writer(tagfile, delimiter='\t')
    
    urlfile = open(file + "urls.csv", "w")
    url_writer = csv.writer(urlfile, delimiter='\t')

    userfile = open(file + "users.csv", "w")
    user_writer = csv.writer(userfile, delimiter='\t')
    tweet_header = ["tid:ID", "lang", "name", "text", "created_at", "full_name",
                        "country", "country_code", "coordinates", "likes", ":LABEL", "eco"]
    tweet_writer.writerow(tweet_header)    
    
    edge_header = [":START_ID", ":END_ID", ":TYPE", "eco"]
    edge_writer.writerow(edge_header)

    tag_header = ["tagid:ID", "hashtag", ":LABEL", "eco"]
    tag_writer.writerow(tag_header)
    
    url_header = ["urlid:ID", "url", ":LABEL", "eco"]
    url_writer.writerow(url_header)

    user_header = ["uid:ID", "screen_name", "name", ":LABEL", "eco"]
    user_writer.writerow(user_header)
    
    validchars = string.ascii_letters + string.digits + ' ' + string.punctuation + " _ÑñáéóíúÁÉÍÓÚ"

    for k, v in hdict["hashtags"].items():
        tag_writer.writerow([v[0], k, "hashtag", v[1]])      
        
    for k, v in hdict["urls"].items():
        url_writer.writerow([v[0], k, "url", v[1]])

    for k, v in hdict["users"].items():
        try:
            user_writer.writerow([k, v[0], v[1], "user", v[2]])      
        except:
            user1 = ''.join(c for c in v[0] if c in validchars)
            user2 = ''.join(c for c in v[1] if c in validchars)
            user_writer.writerow([k, user1, user2, "user", v[2]])  
        
    for v in hdict["tweets"].values():
        try:
            tweet_writer.writerow(v)
        except:
            text = v[3] 
            twt = ''.join(c for c in text if c in validchars)
            tweet_writer.writerow(twt)
            
    for v in hdict["edges"].values():
        edge_writer.writerow(v)
        
    tweetfile.close()
    edgefile.close()
    tagfile.close()
    userfile.close()
    urlfile.close()

In [3]:
hdict = { "hashtags":{}, "urls":{}, "users":{}, "tweets":{}, "edges":{}}

In [4]:
hdict = parse_tweets('data/2018_black_lives_matter.json',  hdict, "BLM")

In [8]:
hdict = parse_tweets('data/2018_me_too.json',  hdict, "MT")

In [10]:
hdict = parse_tweets('data/2018_climate_change.json',  hdict, "CCH")

In [12]:
hdict = parse_tweets('data/2018_gun_control.json',  hdict, "GC")

In [14]:
len(hdict["tweets"])

1226015

In [22]:
hdict3 = hdict.copy()

In [26]:
create_csvs('csvs/', hdict)

In [27]:
!cp csvs/* /usr/share/neo4j/import/

In [ ]:
!/usr/share/neo4j/bin/neo4j-admin import --mode=csv --database=candidatos.db --nodes:tweet /usr/share/neo4j/import/tweets.csv --nodes:user /usr/share/neo4j/import/users.csv  --nodes:hashtag /usr/share/neo4j/import/tags.csv --nodes:url /usr/share/neo4j/import/urls.csv --relationships /usr/share/neo4j/import/edges.csv --id-type=string --delimiter='\t'

In [ ]:
!sudo apt-get install --reinstall neo4j

In [ ]:
!sudo service neo4j start

In [1]:
%reload_ext cypher
%cypher CREATE INDEX ON :user(uid)
%cypher CREATE INDEX ON :url(urlid)
%cypher CREATE INDEX ON :tweet(tid)
%cypher CREATE INDEX ON :hashtag(hid)

0 rows affected.
0 rows affected.
0 rows affected.
0 rows affected.


[]

In [ ]:
#etc/neo4j/conf/neo4j.conf
#dbms.memory.heap.max_size=4096m

In [2]:
import csv
edges = []
with open('data/csvs/edges.csv', 'r') as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    for row in reader:
        if row[2] == 'RETWEETS':
            edges.append([row[1], row[0], row[2]])
        else: 
            edges.append(row)
            
print(len(edges))

30458668


In [3]:
with open('data/csvs/edges2.csv', 'w') as f:
    writer = csv.writer(f, delimiter='\t')
    writer.writerows(edges)

In [4]:
print(len(edges))

30458668
